In [1]:
from mongodb_controller import *

In [3]:
collection_test

Collection(Database(MongoClient(host=['cluster-life-shard-00-02.rbbcd.mongodb.net:27017', 'cluster-life-shard-00-00.rbbcd.mongodb.net:27017', 'cluster-life-shard-00-01.rbbcd.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xh4hp1-shard-0', tls=True), 'database-test'), 'collection-hello')

In [4]:
collection_test.find_one()

{'_id': ObjectId('67962c2396b275f4a7899cb3'),
 'name': 'Test Fund',
 'type': 'Equity',
 'timestamp': '2025-01-26',
 'value': 1000.5}

In [20]:
DATA_SAMPLE = {
    'name': 'hello',
    'type': 'test',
    'timestamp': '2025-01-26',
    'value': 1000.0
}

DATA_SAMPLE_ANOTHER = {
    'name': 'hello',
    'type': 'test',
    'timestamp': '2025-01-26',
    'value': 1000.1
}


In [24]:
collection_test.insert_one(DATA_SAMPLE)

DuplicateKeyError: E11000 duplicate key error collection: database-test.collection-hello index: _id_ dup key: { _id: ObjectId('679636c20524897f24c9e84d') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: database-test.collection-hello index: _id_ dup key: { _id: ObjectId('679636c20524897f24c9e84d') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('679636c20524897f24c9e84d')}}

In [25]:
data = list(collection_test.find())
data

[{'_id': ObjectId('67962c2396b275f4a7899cb3'),
  'name': 'Test Fund',
  'type': 'Equity',
  'timestamp': '2025-01-26',
  'value': 1000.5},
 {'_id': ObjectId('679635870524897f24c9e84b'),
  'name': 'hello',
  'type': 'test',
  'timestamp': '2025-01-26',
  'value': 1000.0},
 {'_id': ObjectId('679635d40524897f24c9e84c'),
  'name': 'hello',
  'type': 'test',
  'timestamp': '2025-01-26',
  'value': 1000.0},
 {'_id': ObjectId('679636c20524897f24c9e84d'),
  'name': 'hello',
  'type': 'test',
  'timestamp': '2025-01-26',
  'value': 1000.0}]

In [18]:
def is_data_in_collection(data, collection):
   return collection.find_one(data) is not None

In [26]:
is_data_in_collection(DATA_SAMPLE_ANOTHER, collection_test)

False

In [29]:
def count_duplicate_data(data, collection):
    query = data.copy()
    query.pop('_id', None)  # _id 필드 제거
    return collection.count_documents(query)

In [30]:
count_duplicate_data(DATA_SAMPLE, collection_test)

3

In [31]:
def remove_duplicates_except_nth(data, collection, n=-1):
   query = data.copy()
   query.pop('_id', None)
   
   if collection.count_documents(query) > 1:
       matching_docs = list(collection.find(query))
       deleted_count = 0
       if abs(n) <= len(matching_docs):
           for doc in matching_docs:
               if doc != matching_docs[n]:
                   result = collection.delete_one({'_id': doc['_id']})
                   deleted_count += result.deleted_count
       return deleted_count
   return 0

In [32]:
remove_duplicates_except_nth(DATA_SAMPLE, collection_test)

2

In [33]:
count_duplicate_data(DATA_SAMPLE, collection_test)

1

In [34]:
list(collection_test.find())

[{'_id': ObjectId('67962c2396b275f4a7899cb3'),
  'name': 'Test Fund',
  'type': 'Equity',
  'timestamp': '2025-01-26',
  'value': 1000.5},
 {'_id': ObjectId('679636c20524897f24c9e84d'),
  'name': 'hello',
  'type': 'test',
  'timestamp': '2025-01-26',
  'value': 1000.0}]